In [140]:
import os
import osmnx as ox
import networkx as nx

import pandas as pd
import geopandas as gpd

from lxml import etree
from bs4 import BeautifulSoup

In [4]:
osm_path = r'C:\Users\Wei Zhou\OneDrive - National University of Singapore\Pesearch Project\Traffic dataset\Singapore Geodatabase\OSM\singapore_osm\singapore-2024-07-02.osm'

In [5]:
tree = etree.parse(osm_path)

In [137]:
def get_relation_member(tree, osm_id):
    
    element = tree.getroot()

    # find the root "relation" based on osm id
    # xpath parser <tag_name attribute=value/>
    relation = element.findall(f"relation[@id='{osm_id}']")
    # assert len(relation) == 1, 'The relation id for PT system should be unique, but found ' + str(len(relation))
    # the "relation" should be unique
    relation = relation[0]
    
    relation_string = etree.tostring(relation, pretty_print=False)
    
    soup = BeautifulSoup(relation_string, 'xml')
    # members of "relation"
    member = []
    for m in soup.find_all('member'):
        if not(m.get('ref', None) is None):
            member.append(
                {'osm_id' : m.get('ref', None),
                 'type'   : m.get('type', None) })
    
    # tags of "relation", contain information
    info = {
        'osm_id'        : osm_id,
        'member_osm_id' : [v['osm_id'] for v in member],
        'member_type'   : [v['type'] for v in member], 
        'member_number' : len(member)}
    
    for t in soup.find_all('tag'):
        if not(t.get('k', None) is None):
            info[t.get('k', 'Null')] = t.get('v', 'Null')
    
    return info
# ======================================================================================


In [131]:
pt_system_osm_id = "3149677"

info = get_relation_member(tree, osm_id=pt_system_osm_id, member_osm_type='relation')

member_info = {}
for member_osm_id, member_osm_type in zip(info['member_osm_id'].split(','), info['member_osm_type'].split(',')):
    member_info[member_osm_id] = get_relation_member(tree, member_osm_id, member_osm_type)

TypeError: get_relation_member() got an unexpected keyword argument 'member_osm_type'

In [124]:
def retrieve_metro_relation(xml_tree, osm_id):
    '''
    
    '''
    

In [245]:
def _add_tag_info(bs4soup, detail):

    detail['detail'] = {}
    
    for elem in bs4soup.find_all('tag'):
        if not( elem.get('k', None) is None ):
            detail['detail'][elem.get('k', None)] = elem.get('v', None)
    
    return detail
# ======================================================================
def _parser_osm_relation_detail(bs4soup):
    detail = {'member' : []} 
    
    order_ix = 0
    for elem in bs4soup.find_all('member'):
        if not( elem.get('ref', None) is None ) :
            detail['member'].append(
                (elem.get('ref'), {
                    'ix'   : order_ix,
                    'type' : elem.get('type', None) } )
            )
            order_ix = order_ix + 1

    # detail['member_number'] = len(detail['member'])
    
    # tags
    detail = _add_tag_info(bs4soup, detail)
    
    return detail
# =============================================================================
def _parser_osm_relation_route_detail(bs4soup):
    
    detail = { 'member' : {
        'node' : [],
        'way'  : [] }}

    # node
    order_node_ix = 0
    order_way_ix = 0
    for elem in bs4soup.find_all('member'):
        if not( elem.get('ref', None) is None ) and not (elem.get('type', None) is None) :
            type = elem.get('type', None) 
            
            if type == 'node':
                detail['member']['node'].append(
                    (elem.get('ref'), {
                    'ix'   : order_node_ix,
                    'role' : elem.get('role', None) } )
                )
                order_node_ix = order_node_ix + 1
                
            elif type == 'way':
                detail['member']['way'].append(
                    (elem.get('ref'), {
                        'ix'   : order_way_ix,
                        'role' : elem.get('role', None) } ) 
                )
                order_way_ix = order_way_ix + 1
            
            else:
                continue
                
    # detail['node_number'] = len(detail['member']['node'])
    # detail['way_number']  = len(detail['member']['way'])
    
    # tags
    detail = _add_tag_info(bs4soup, detail)

    return detail
# =============================================================================
def _parser_route_stop_node(bs4soup):

    bs4soup = bs4soup.find('node')
    
    detail = {
        'lat' : bs4soup.get('lat', None),
        'lon' : bs4soup.get('lon', None)  }
    
    # tags
    detail = _add_tag_info(bs4soup, detail)
    
    return detail
# =============================================================================
def _parser_way(bs4soup):
    
    detail = {'node' : [] }
    
    ix = 0
    for elem in bs4soup.find_all('nd'):
        detail['node'].append( (
            elem.get('ref'), {
            'ix'  : ix })
        )
        ix = ix + 1
    
    # tags
    detail = _add_tag_info(bs4soup, detail)
    
    return detail
# =============================================================================
def _parser_way(bs4soup):

    bs4soup = bs4soup.find('node')
    
    detail = {
        'lat' : bs4soup.get('lat', None),
        'lon' : bs4soup.get('lon', None)  }
    
    return detail
# =============================================================================
def _get_xml_bs4soup(xml_tree, osm_id, dtype):
    
    element = tree.getroot()
    # find the root "relation" based on osm id
    # xpath parser <tag_name attribute=value/>
    relation = element.findall(f"{dtype}[@id='{osm_id}']")
    # assert len(relation) == 1, 'The relation id for PT system should be unique, but found ' + str(len(relation))
    # the "relation" should be unique
    relation = relation[0]
    
    relation_string = etree.tostring(relation, pretty_print=False)

    soup = BeautifulSoup(relation_string, 'xml')
    
    return soup
# =============================================================================
def parser_osm_detail(bs4soup, parser_type):
    if parser_type == 'relation':
        detail = _parser_osm_relation_detail(soup)
    elif parser_type == 'route':
        detail = _parser_osm_relation_route_detail(soup)
    elif parser_type == 'stop_node':
        detail = _parser_route_stop_node(soup)
    elif parser_type == 'way':
        detail = _parser_way(soup)
    else:
        return None
    return detail
# =============================================================================
def _decide_parser_type(bs4soup):
    
    for child in bs4soup.children:
        dtype = child.name
        break
    
    if dtype == 'relation':
        if child.get('type', None) == 'route':
            parser_type = 'route'
        else:
            parser_type = 'relation'
            
    elif dtype == 'node':
        parser_type = 'stop_node'
    elif dtype == 'way':
        parser_type = 'way'
        
    return parser_type

```
--relation
----relation
------relation
--------route
----------node
----------way
------------node
```

In [247]:


osm_id = '3149677'
dtype = 'relation'

structure = nx.DiGraph()

soup = _get_xml_bs4soup(xml_tree=tree, osm_id=osm_id, dtype=dtype)
parser_type = _decide_parser_type(soup)
print(parser_type)

detail = parser_osm_detail(soup, parser_type)


graph = add_children_node(structure, osm_id, detail, parser_type)

relation


In [246]:
def add_children_node(graph, parent_node, detail, parser_type):

    graph.add_node(parent_node, **detail['detail'])
    
    if parser_type == 'relation':
        graph.add_nodes_from(detail['member'])
        for node in detail['member']:
            graph.add_edge(parent_node, node[0])
    
    elif parser_type == 'route':
        graph.add_nodes_from(detail['member']['node'])
        for node in detail['member']:
            graph.add_edge(parent_node, node[0])

        graph.add_nodes_from(detail['member']['way'])
        for node in detail['member']:
            graph.add_edge(parent_node, node[0])
    
    return graph

In [248]:
graph.nodes(data=True)

NodeDataView({'3149677': {'name': 'Singapore Public Transportation', 'network': 'Singapore', 'type': 'network'}, '1152539': {'ix': 0, 'type': 'relation'}, '3149676': {'ix': 1, 'type': 'relation'}})